In [2]:
%load_ext watermark


In [3]:
%watermark

Last updated: 2022-08-15T19:07:33.658443-04:00

Python implementation: CPython
Python version       : 3.8.9
IPython version      : 8.4.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [7]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

In [8]:
%watermark --iversions

pandas : 1.4.3
numpy  : 1.23.1
sklearn: 1.1.2



In [3]:
data = pd.read_csv('../input/creditcard.csv.gz', compression='gzip')

In [4]:
data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [5]:
features = data.columns[:-1]
target = data.Class.values

In [8]:
train = data.copy()

In [12]:
%%time

train_oof = np.zeros((train.shape[0],))

n_fold = 5 



kf = StratifiedKFold(n_splits=n_fold, random_state=137, shuffle=True)


for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = target[idx_tr]
    y_va = target[idx_va]

    print(f'Fold: {fold}')
    clf = HistGradientBoostingClassifier()
    clf.fit(X_tr, y_tr)
    val_predictions = clf.predict_proba(X_va)[:,1]
    train_oof[idx_va] = val_predictions
    score = roc_auc_score(y_va, val_predictions)
    print(f'Fold: {fold} - score {score:.2}')

Fold: 0
Fold: 0 - score 0.29
Fold: 1
Fold: 1 - score 0.4
Fold: 2
Fold: 2 - score 0.66
Fold: 3
Fold: 3 - score 0.28
Fold: 4
Fold: 4 - score 0.74
CPU times: user 15 s, sys: 17.6 s, total: 32.6 s
Wall time: 6.69 s


In [13]:
roc_auc_score(target, train_oof)

0.4671539954324679

In [55]:
%%time

train_oof = np.zeros((train.shape[0],))

n_fold = 5 



kf = StratifiedKFold(n_splits=n_fold, random_state=137, shuffle=True)


for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = target[idx_tr]
    y_va = target[idx_va]

    print(f'Fold: {fold}')
    clf = HistGradientBoostingClassifier(early_stopping=None, max_iter=420, max_depth=2, 
                                         random_state=137, max_bins=250)
    clf.fit(X_tr, y_tr)
    val_predictions = clf.predict_proba(X_va)[:,1]
    train_oof[idx_va] = val_predictions
    score = roc_auc_score(y_va, val_predictions)
    print(f'Fold: {fold} - score {score:.4}')

Fold: 0
Fold: 0 - score 0.9083
Fold: 1
Fold: 1 - score 0.9536
Fold: 2
Fold: 2 - score 0.731
Fold: 3
Fold: 3 - score 0.8575
Fold: 4
Fold: 4 - score 0.8984
CPU times: user 1min 30s, sys: 1min 37s, total: 3min 8s
Wall time: 24.5 s


In [52]:
roc_auc_score(target, train_oof)

0.8570062598037304

In [46]:
0.8570062598037304

0.8570062598037304